### Harris

1. 基于局部信号的自相关函数.当一个窗口在图像中滑动时:
    1. 如果滑过某块区域,窗口内灰度值变化不剧烈,这个是平坦区域.
    2. 如果滑过边缘的正交方向,窗口内灰度值变化剧烈.这是个边缘.
    3. 当无论朝着哪个方向滑动,窗口内灰度值变化都剧烈.那这是个角点.

2. 自适应函数

$$c(x,y)=\sum_{(u,v)\in W(x,y)}w(u,v)[I(u,v)-I(u+\delta u,v+ \delta v)]^2$$

w(u,v)是加权函数,可以是常数,也可以是高斯加权函数.

利用泰勒一阶展开,得到一个自相关矩阵

$$\begin{bmatrix}
a & b\\ 
c & d
\end{bmatrix}=\begin{bmatrix}
\sum_{u} (I_x(u,v))^2 & \sum_{u} (I_x(u,v)I_y(u,v)) & \\ 
\sum_{u} (I_x(u,v)I_y(u,v))  & \sum_{u}(I_y(u,v))^2  & 
\end{bmatrix}$$

3. lamda1 和lamda2是自相关矩阵的两个特征值
    1. lamda1和lamda2都很小时,表示是平坦
    2. lamda1或lamda2有一个很大,一个很小时,表示是边缘.
    3. lamda1和lamda2都很大时,表示的是角点.

4. 求解析解很费时.因此可以求一个lamda1和lamda2的相对大小来描述3中的三个行为.
$$R=\lambda _1\lambda _2-k(\lambda _1+\lambda _2)^2$$

det是行列式,trace是矩阵的迹

$$R=det(C(x,y))-k(trace(C(x,y)))^2$$

相应的有:

$$det(C(x,y))=\lambda_1\lambda_2=ac-b^2$$
$$trace(C(x,y))=\lambda_1+\lambda_2=a+c$$

k取值0.04~0.06间

In [1]:
import os
import sys 
abs_path=os.path.abspath(os.curdir)
cvlib_path=abs_path+"/../cvlib/"
sys.path.append(cvlib_path)
from convs import *
from sobel import *

In [2]:
def harris():
    dx,dy=getSobelKernel(3)
    print(dx)
    print(dy)

In [3]:
harris()

[[ 1.  2.  1.]]
[[ 1.  0. -1.]]
kernel and image should be square
kernel and image should be square
None
None
